In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, results as rs

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=1700851321008) was called 1 times


2021-01-07 15:50:52,727 - NumExpr defaulting to 8 threads.


In [5]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=1700851321872) was called 1 times


In [6]:
fa,unmapped, end_time = feat.preprocess_faults(scs_raw)

#fa= fa[fa['timestamp']<(fa['timestamp'].max()-pd.to_timedelta(7, unit='D'))]

Running: Function "preprocess_faults" (id=1700851322016) was called 1 times
Running: Function "load_tote_lookup" (id=1700851322592) was called 1 times
Running: Function "load_module_lookup" (id=1700848356368) was called 1 times
Running: Function "load_ID_lookup" (id=1700851319280) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [7]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [8]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights = [0.7,0.3],duration_thres=0)

df = feat.log_totes(df) 

Running: Function "load_PTT_lookup" (id=1700851321296) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1700851321728) was called 1 times
Running: Function "load_PTT_lookup" (id=1700851321296) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1700851321728) was called 2 times
Running: Function "load_PTT_lookup" (id=1700851321296) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1700851321728) was called 3 times
Running: Function "load_PTT_lookup" (id=1700851321296) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1700851321728) was called 4 times
Running: Function "load_PTT_lookup" (id=1700851321296) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1700851321728) was called 5 times
Running: Function "load_PTT_lookup" (id=1700851321296) was called 6 times
Previous Hours Weighted
Running

In [9]:
df_week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]

In [10]:
X,y = md.gen_feat_var(df_week,target = 'Availability', features = ['Faults','Totes'])


In [11]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 10)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


KeyboardInterrupt: 

In [ ]:
R2_OOS

In [ ]:
Output = rs.create_output(fa_PTT,Coeff,end_time)

# Plotting

In [ ]:
Output['Total'] = Output[:][Output.columns[8:]].sum(axis=1)

In [ ]:
Total = Output[['Asset Code','Total']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(y='Total',x = 'Asset Code',data = Total.sort_values(by='Total',ascending=False).head(10),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
PTT081 = Output[['Asset Code','PTT081']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(y='PTT081',x = 'Asset Code',data = PTT081.sort_values(by='PTT081',ascending=False).head(5),color = 'r')
plt.ylabel('Lost Singles')
plt.title('PTT081 Lost Singles Last 12 Hours')

In [ ]:
Total_colour = Output[['Tote Colour','Total']].groupby('Tote Colour').sum().reset_index()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(y='Total',x = 'Tote Colour',data = Total_colour.sort_values(by='Total',ascending=False),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
Total_colour

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df_week, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df_week,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df_week, line_kws={"color": "red"})
plt.xlim(0, 5)

In [15]:
Shift = [0,0,0,10,10,10,20,20,20]
Weights = [[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1]]

In [ ]:
Weights[8]

In [ ]:
y

In [ ]:
at

In [ ]:
fa_floor

In [19]:
df.drop(['Availability','Blue Tote Loss','Grey Tote Loss','timestamp'],axis=1)

,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,C0501STA201,C0501STA217,C0501STA219,C0501STA228,...,SCS202,SCS203,SCS204,SCS205,SCS206,SCSM20,C1404RDC146,C1404STA149,C14PTT202,log_totes
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.591024
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.482777
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.162658
3,0.0,0.0,0.0,0.0,2.062137,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.512540
4,0.0,0.0,0.0,0.0,0.883773,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.494485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.741268,0.741172,0.000000,3.916931,0.0,3.385884,0.0,0.000000,0.317295,3.936716
462,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.317686,1.058899,0.741247,4.235067,0.0,3.597478,0.0,0.000000,0.000000,4.332048
463,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.741328,0.317680,1.058880,2.753261,0.0,1.269741,0.0,0.741301,2.960411,4.177204
464,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.317712,0.000000,1.058981,2.117555,0.0,0.000000,0.0,0.317700,2.008558,4.054390
